In [10]:
import collections
import functools
import itertools
import math
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import networkx as nx
import operator
import pprint as pp
import pyvis
import random
import random
import regex
import statistics


plt.rc('image', cmap='viridis')
plt.style.use('dark_background')


def Input(day):
    'Open input file for the day'
    file = '2023/input{0}.txt'.format(day)
    return open(file)

def strip_lines(f):
    return [l.strip() for l in f.readlines()] 


## Day 1: Trebuchet?!

In [11]:
with Input(1) as f:
    lines = strip_lines(f)

def calibrate_one(line):
    digits = regex.findall('\d', line)
    return int(digits[0]) * 10 + int(digits[-1])

def calibrate_two(line):
    digits = regex.findall('\d|one|two|three|four|five|six|seven|eight|nine', line, overlapped=True)
    return word_to_int(digits[0]) * 10 + word_to_int(digits[-1])

def word_to_int(word):
    match word:
        case 'one':
            return 1
        case 'two':
            return 2
        case 'three':
            return 3
        case 'four':
            return 4
        case 'five':
            return 5
        case 'six':
            return 6
        case 'seven':
            return 7
        case 'eight':
            return 8
        case 'nine':
            return 9
        case _:
            return int(word)

print('Part One: ', sum(map(calibrate_one, lines)))
print('Part Two: ', sum(map(calibrate_two, lines)))


Part One:  55172
Part Two:  54925


## Day 2: Cube Conundrum

In [12]:
with Input(2) as f:
    lines = strip_lines(f)

def parse_game(line):
    ''' 
    Find all the numbers for each color and return a tuple
    of ([red], [green], [blue], game_number)
    '''
    game_number = int(regex.match(r'Game (\d+)', line)[1])
    red = list(map(int, regex.findall(r'(\d+) red', line)))
    green = list(map(int, regex.findall(r'(\d+) green', line)))
    blue = list(map(int, regex.findall(r'(\d+) blue', line)))
    game = (red, green, blue, game_number)
    return game


def possible(game):
    #12 red cubes, 13 green cubes, and 14 blue cubes
    red = all([True if red <= 12 else False for red in game[0]])
    green = all([True if green <= 13 else False for green in game[1]])
    blue = all([True if blue <= 14 else False for blue in game[2]])
    return red and green and blue

games = [parse_game(line) for line in lines]

print('Part One: ', sum([game[3] for game in games if possible(game)]))
print('Part Two: ', sum(max(game[0]) * max(game[1]) * max(game[2]) for game in games))


Part One:  2278
Part Two:  67953


## Day 3: Gear Ratios

In [13]:
with Input(3) as f:
    schematic = strip_lines(f)

MAX = len(schematic)

# vectors for adjacent cells
neighbor_indexes = {(dy, dx) for dx,dy in itertools.permutations([-1,-1,0,1,1], 2)}

def find_potential_parts(schematic):
    '''
    Find the x ranges of all parts, indexed by y
    [[part, ...], ...]
    '''
    return [ list(regex.finditer('\d+', line)) for line in schematic]

def neighbors(schematic):
    '''
    Return the value of all neighboring cells
    '''
    return lambda i : [schematic[i[0] + dy][i[1] + dx] for dy,dx in neighbor_indexes if i[0]+ dy >= 0 and i[0]+ dy < MAX and i[1]+ dx >= 0 and i[1]+ dx < MAX]

def neighbor_locations(schematic):
    '''
    Return the indexes of all neighboring cells
    '''
    return lambda i : [(i[0] + dy, i[1] + dx) for dy,dx in neighbor_indexes if i[0]+ dy >= 0 and i[0]+ dy < MAX and i[1]+ dx >= 0 and i[1]+ dx < MAX]

get_neighbors = neighbors(schematic)
get_neighbor_indexes = neighbor_locations(schematic)

def valid_part(y, part):
    '''
    Does the part a y, part_x_range neighbor a symbol?
    '''
    part_range = range(part.start(), part.end())

    for x in part_range:
        neighbors =  get_neighbors((y, x))
        for neighbor in neighbors:
            if neighbor not in '0123456789.':
                return True
    return False

def find_potential_gears(schematic):
    '''
    Find the (y,x) coordinates of all * characters
    '''
    return [(y,x) for y, line in enumerate(schematic) for x, i in enumerate(line) if i == '*']

def find_neighboring_part_numbers(gear, parts_list):
    '''
    Find all the part numbers that neighbor a potential gear, valid gears will only have two neighboring parts
    '''
    neighboring_part_numbers = []
    gear_neighbors = get_neighbor_indexes(gear)
    for y, parts in enumerate(parts_list):
        for part in parts:
            part_range = range(part.start(), part.end())
            for x in part_range:
                if (y,x) in gear_neighbors:
                    neighboring_part_numbers.append(int(part.group()))
                    break
    return neighboring_part_numbers

# A bunch of junk follows

potential_parts = find_potential_parts(schematic)
valid_parts = [[part for part in part_list if valid_part(y, part)] for y, part_list in enumerate(potential_parts)]

valid_part_sum = sum([int(part.group()) for parts in valid_parts for part in parts])

potential_gears = find_potential_gears(schematic)
potential_gear_neighboring_part_numbers = [find_neighboring_part_numbers(gear, valid_parts) for gear in potential_gears]

valid_gear_neighboring_part_numbers = list(filter(lambda ls: len(ls) == 2, potential_gear_neighboring_part_numbers))

gear_values = [functools.reduce(operator.mul, part_numbers, 1) for part_numbers in valid_gear_neighboring_part_numbers]
valid_gear_sum = sum(gear_values)

print('Part One: ', valid_part_sum)
print('Part Two: ', valid_gear_sum)


Part One:  509115
Part Two:  75220503


## Day 4: Scratchcards 

In [14]:
with Input(4) as f:
    lines = strip_lines(f)

Card = collections.namedtuple('Card', ['card_number', 'winning_numbers', 'numbers_you_have', 'value', 'matches', 'copies'])

def parse_card(line):
    '''
    Turn a line into a Card
    '''
    card_number = int(regex.match(r'Card +(\d+):', line).group(1))
    winning_numbers = set(map(int, regex.findall(r':(.*)\|', line)[0].split()))
    numbers_you_have = set(map(int, regex.findall(r'\|(.*)', line)[0].split()))

    return Card(card_number, winning_numbers, numbers_you_have, 0, 0, 1)

def assess_value(card):
    '''
    Calculate the value of a card, as well as the number of matches
    '''
    matches = len(card.winning_numbers.intersection(card.numbers_you_have))
    winner = 1 if matches != 0 else 0
    return card._replace(value=2 ** (matches - 1) * winner, matches= matches)


# parse all the cards
cards = [parse_card(line) for line in lines]

#value all the cards
valued_cards = list(map(assess_value, cards))

# sum the values of the cards
part_one = sum([card.value for card in valued_cards])

# for each card, add to cards # of copies as necessary
for i, card in enumerate(valued_cards):
    # add this cards number of copies to subsequent cards based on the number of matches
    for idx in range(i+1, i+1+card.matches):
        valued_cards[idx] = valued_cards[idx]._replace(copies=valued_cards[idx].copies+card.copies)

part_two = sum([card.copies for card in valued_cards])


print('Part One: ', int(part_one))
print('Part Two: ', part_two)


Part One:  21088
Part Two:  6874754


## Day 5: If You Give A Seed A Fertilizer

In [15]:
with Input(5) as f:
    text = f.read()

# Stole this recipe rather than update to python3.12
def batched(iterable, n):
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(itertools.islice(it, n)):
        yield batch


def map_to_range(i, range_maps):
    for range_map in range_maps:
        if i in range(range_map[1], range_map[1]+range_map[2]):
            return i + range_map[0] - range_map[1]
    return i

def reverse_map_to_range(i, range_maps):
    for range_map in range_maps:
        if i in range(range_map[0], range_map[0]+range_map[2]):
            return i + range_map[1] - range_map[0]
    return i

def seeds_contain(i, seed_pairs):
    for seed_range in seed_pairs:
        if i in range(seed_range[0], seed_range[0]+seed_range[1]):
            return True
    return False

seeds = list(map(int,regex.findall(r'seeds: ((?:\d+ ?)+)', text)[0].split()))

seed_to_soil = list(map(int,regex.findall(r'seed-to-soil map:\n((?:\d+[ \n]?)+)', text)[0].split()))
seed_to_soil = list(batched(seed_to_soil, 3))

soil_to_fertilizer = list(map(int,regex.findall(r'soil-to-fertilizer map:\n((?:\d+[ \n]?)+)', text)[0].split()))
soil_to_fertilizer = list(batched(soil_to_fertilizer, 3))

fertilizer_to_water = list(map(int,regex.findall(r'fertilizer-to-water map:\n((?:\d+[ \n]?)+)', text)[0].split()))
fertilizer_to_water = list(batched(fertilizer_to_water, 3))

water_to_light = list(map(int,regex.findall(r'water-to-light map:\n((?:\d+[ \n]?)+)', text)[0].split()))
water_to_light = list(batched(water_to_light, 3))

light_to_temperature = list(map(int,regex.findall(r'light-to-temperature map:\n((?:\d+[ \n]?)+)', text)[0].split()))
light_to_temperature = list(batched(light_to_temperature, 3))

temperature_to_humidity = list(map(int,regex.findall(r'temperature-to-humidity map:\n((?:\d+[ \n]?)+)', text)[0].split()))
temperature_to_humidity = list(batched(temperature_to_humidity, 3))

humidity_to_location = list(map(int,regex.findall(r'humidity-to-location map:\n((?:\d+[ \n]?)+)', text)[0].split()))
humidity_to_location = list(batched(humidity_to_location, 3))

min_location = float('inf')
for seed in seeds:
    soil = map_to_range(seed, seed_to_soil)
    fertilizer = map_to_range(soil, soil_to_fertilizer)
    water = map_to_range(fertilizer, fertilizer_to_water)
    light = map_to_range(water, water_to_light)
    temperature = map_to_range(light, light_to_temperature)
    humidity = map_to_range(temperature, temperature_to_humidity)
    location = map_to_range(humidity, humidity_to_location)
    if location < min_location:
        min_location = location

print('Part One: ', min_location)

'''
This takes forever.
'''
# seed_pairs = list(batched(seeds, 2))

# min_location = float('inf')
# for location in itertools.count(0):
#     humidity = reverse_map_to_range(location, humidity_to_location)
#     temperature = reverse_map_to_range(humidity, temperature_to_humidity)
#     light = reverse_map_to_range(temperature, light_to_temperature)
#     water = reverse_map_to_range(light, water_to_light)
#     fertilizer = reverse_map_to_range(water, fertilizer_to_water)
#     soil = reverse_map_to_range(fertilizer, soil_to_fertilizer)
#     seed = reverse_map_to_range(soil, seed_to_soil)

#     if seeds_contain(seed, seed_pairs):
#         min_location = location
#         break

print('Part Two: ', min_location)


Part One:  227653707
Part Two:  227653707


## Day 6: Wait For It

In [16]:
with Input(6) as f:
    lines = strip_lines(f)

times = list(map(int,regex.findall(r'Time:(.*)', lines[0])[0].split()))
distances = list(map(int,regex.findall(r'Distance:(.*)', lines[1])[0].split()))

big_time = int(''.join(list(map(str, times))))
big_distance = int(''.join(list(map(str, distances))))
big_race = [(big_time, big_distance)]

races = list(itertools.zip_longest(times, distances))

def multiply_races(races):
    '''
    https://oeis.org/A004247
    '''
    counts = []
    for race in races:
        counts.append(len(list(filter(lambda x: x > race[1], [i * (race[0] - i) for i in range(0, race[0])]))))
    return functools.reduce(operator.mul,counts)

print('Part One:', multiply_races(races))
# Fairly slow
# print('Part Two:', multiply_races(big_race))


Part One: 1195150


# Day 7: Camel Cards

In [17]:
with Input(7) as f:
    lines = strip_lines(f)

hands_and_bids = [ tuple(line.split()) for line in lines]

cards = "AKQJT98765432"
values= [0xE, 0xD, 0xC, 0xB, 0xA, 0x9, 0x8, 0x7, 0x6, 0x5, 0x4, 0x3, 0x2]

card_values = dict(zip(cards, values))


def rank_hand(hand_and_bid):
    sorted_hand = sorted(hand_and_bid[0])
    type = 1
    kinds = []
    for _, g in itertools.groupby(sorted_hand):
        kinds.append(len(list(g)))
    if 5 in kinds:
        type = 5
    elif 4 in kinds:
        type = 4
    elif 3 in kinds:
        if 2 in kinds:
            type = 3.5
        else:
            type = 3
    elif 2 in kinds:
        if 2 == len([kind for kind in kinds if kind == 2]):
            type = 2.5
        else:
            type = 2
    return [type]  + [card_values[card] for card in hand_and_bid[0]]

sorted_hands_and_bids = sorted(hands_and_bids, key=rank_hand)
winnings = 0
for i, (hand, bid) in enumerate(sorted_hands_and_bids):
    winnings += int(bid) * (i + 1)

print('Part One:', winnings)

card_values['J'] = 0x1

def rank_hand_with_jokers(hand_and_bid):
    sorted_hand = sorted(hand_and_bid[0])
    type = 1
    kinds = []
    num_jokers = len([card for card in sorted_hand if card == 'J'])
    kinds.append(0) # Make sure it's not empty in the case of JJJJJ
    for _, g in itertools.groupby([card for card in sorted_hand if card != 'J']):
        kinds.append(len(list(g)))

    kinds = sorted(kinds, reverse=True)
    if kinds[0] + num_jokers >= 5:
        type = 5
    elif kinds[0] + num_jokers == 4:
        type = 4
    elif kinds[0] + num_jokers >= 3:
        if kinds[1] == 2:
            type = 3.5
        else:
            type = 3
    elif kinds[0] + num_jokers == 2:
        if kinds[1] == 2:
            type = 2.5
        else:
            type = 2
    return [type]  + [card_values[card] for card in hand_and_bid[0]]

sorted_hands_and_bids = sorted(hands_and_bids, key=rank_hand_with_jokers)
winnings = 0
for i, (hand, bid) in enumerate(sorted_hands_and_bids):
    winnings += int(bid) * (i + 1)

print('Part Two:', winnings)


Part One: 248217452
Part Two: 245576185


# Day 8: Haunted Wasteland

In [36]:
with Input(8) as f:
    lines = strip_lines(f)

instructions = lines[0]

# Build network graph
network = dict()
for line in lines[2:]:
    matches = regex.findall(r'[A-Z0-9]{3}', line)
    network[matches[0]] = {'L': matches[1], 'R': matches[2]}

def solver(node, endFn):
    '''
    Apply the full set of instructions until the endFn is satisfied
    '''
    current = node
    count = 0
    while endFn(current):
        for instruction in instructions:
            current = network[current][instruction]
            count += 1
    return count

endsZ = lambda x: x[-1] != 'Z' # Ends in 'Z'
isZZZ = lambda x: x != 'ZZZ' # is 'ZZZ'

start_nodes = [node for node in network if node[-1] == 'A']
solve_lengths = [solver(node, endsZ) for node in start_nodes]

print('Part One:', solver('AAA', isZZZ))
print('Part Two:', math.lcm(*solve_lengths))

Part One: 17287
Part Two: 18625484023687


# Day 9: Mirage Maintenance

In [105]:
with Input(9) as f:
    lines = strip_lines(f)

lines = [line.split() for line in lines]
sequences = [[int(i) for i in line] for line in lines]


def next_value(seq):
    diffs = collections.deque([seq])
    while True:
        diffs.appendleft([b - a for (a,b) in itertools.pairwise(diffs[0])])
        if not any(diffs[0]):
            break

    for i, diff in enumerate(diffs):
        if i == len(diffs) - 1:
            break
        diffs[i+1].append(diffs[i+1][-1] + diff[-1])
    
    return diffs[-1][-1]

def prev_value(seq):
    diffs = collections.deque([collections.deque(seq)])
    while True:
        diffs.appendleft(collections.deque([b - a for (a,b) in itertools.pairwise(diffs[0])]))
        if not any(diffs[0]):
            break

    for i, diff in enumerate(diffs):
        if i == len(diffs) - 1:
            break
        diffs[i+1].appendleft(diffs[i+1][0] - diff[0])
        
    return diffs[-1][0]

part_one = sum([next_value(seq) for seq in sequences])
part_two = sum([prev_value(seq) for seq in sequences])
print('Part One:', part_one)
print('Part Two:', part_two)

Part One: 1980437560
Part Two: 977
